# Function Call

## Demo code to make function calls
A quick demo code to make function calls. 

Please ensure you have *.env* file in your HOME/Documents/src/openai/ folder

In [ ]:
from openai import AzureOpenAI
from dotenv import load_dotenv
from pprint import pprint
import pandas as pd
import json
import os

env_path = os.getenv("HOME") + "/Documents/src/openai/.env"
load_dotenv(dotenv_path=env_path, verbose=True)

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://pvg-azure-openai-uk-south.openai.azure.com"

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2023-05-15"
)

In [ ]:
df = pd.read_csv("data/employees.csv")

def get_colleague_info(colleague_name: str):
    df1 = df[df.name == colleague_name.lower()]
    if df1.empty:
        return None
    else:
        colleague_info = {
            "name": df1.name.loc[df1.index[0]],
            "role": df1.role.loc[df1.index[0]]
        }
        return json.dumps(colleague_info)

In [ ]:
tools = [
    {"type": "function", 
         "function": {
            "name": "get_colleague_info", 
            "description": "Get colleagues' infomration",
            "parameters": {
                "type":"object",
                "properties": {
                    "colleague_name": {
                        "type": "string", 
                        "description": "the name of the colleague"
                    }
                }
            }
        }
    }
]

In [ ]:
system_prompt = "You are a helpful assistant."
user_prompt = "what's the role of my colleague Michael, Hunter and Jack?"
messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
           ]

response = client.chat.completions.create(
    model="gpt-35-turbo", 
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    tools=tools, 
    tool_choice = 'auto'
)

response_message = response.choices[0].message

if response.choices[0].finish_reason == "tool_calls":
    print("GPT asked us to call a function.")

    messages.append(response_message)

    for tool_call in response.choices[0].message.tool_calls: 
        function_name = tool_call.function.name
        params = json.loads(tool_call.function.arguments)
    
        function_response = get_colleague_info (
            **params
        )

        messages.append({"role": "tool", "tool_call_id": tool_call.id, "name": function_name, "content": function_response})
        
    second_response = client.chat.completions.create(
        model="gpt-35-turbo", 
        messages = messages,
    )
        
    print(second_response.choices[0].message.content)



In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def get_colleague_info_enh(colleague_name: str):

    """Get the information of a colleague.

    Args:
        colleague_name: name of the colleague
    """
    
    df1 = df[df.name == colleague_name.lower()]
    if df1.empty:
        return None
    else:
        colleague_info = {
            "name": df1.name.loc[df1.index[0]],
            "role": df1.role.loc[df1.index[0]]
        }
        return json.dumps(colleague_info)

print(json.dumps(convert_to_openai_tool(get_colleague_info_enh), indent=2))

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain.agents import Tool, AgentExecutor, create_openai_tools_agent
from langchain import hub
 
model = AzureChatOpenAI(deployment_name="gpt-35-turbo")

tools = [
    Tool(
        name = "get_colleague_info_enh",
        func = get_colleague_info_enh.run,
        description = "useful for colleague information extraction"
    )
]

prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_openai_tools_agent(model, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=False, handle_parsing_errors=True
)
agent_executor.invoke({"input": "what's the role of my colleague Michael, Hunter and Jack?"})

